In [2]:
import json
import requests
import re, time
import numpy as np
import time, datetime
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
def retrieve_html(url):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string): 

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """
    response = requests.get(url)
    return (response.status_code, response.text)
    pass

In [4]:
code, text = retrieve_html("https://en.wikipedia.org/wiki/List_of_Carnegie_Mellon_University_people")

In [5]:
soup = BeautifulSoup(text,'html.parser')
next_page = set()
for link in soup.find('div',{'id':'bodyContent'}).find_all('a', href=True):
    tar_url=link.attrs['href']
    if tar_url.startswith('/wiki/') and tar_url.find(':') == -1:
        next_page.add(tar_url)
people_list = []
for url in tqdm(next_page):
    time.sleep(0.1)
    code, new_text = retrieve_html("https://en.wikipedia.org"+url)
    if (new_text.find('<th scope="row">Born</th>') != -1 and new_text.find('Carnegie_Mellon_University')):
        print(url)
        people_list.append(url)
print(len(people_list))

  0%|          | 0/1065 [00:00<?, ?it/s]

/wiki/Den%C3%A9e_Benton


  0%|          | 2/1065 [00:00<03:50,  4.61it/s]

/wiki/Katy_Mixon


  0%|          | 5/1065 [00:01<04:06,  4.30it/s]

/wiki/Stephanie_Kwolek


  1%|          | 8/1065 [00:01<04:00,  4.40it/s]


KeyboardInterrupt: 

In [52]:
test_url = 'https://en.wikipedia.org/wiki/Andy_Warhol'
code, html = retrieve_html(test_url)
soup = BeautifulSoup(html,'html.parser')
div = soup.find('div', {"class": "mw-parser-output"})

In [53]:
text = ""
for p in div.find_all("p", recursive=False):
    tmp = re.sub("(\[[0-9]+\])", "", p.text)
    tmp = re.sub("(\(.*\))", "", tmp).rstrip().strip()
    text += tmp
print(text)

Andy Warhol .Born and raised in Pittsburgh, Warhol initially pursued a successful career as a commercial illustrator. After exhibiting his work in several galleries in the late 1950s, he began to receive recognition as an influential and controversial artist. His New York studio, The Factory, became a well-known gathering place that brought together distinguished intellectuals, drag queens, playwrights, Bohemian street people, Hollywood celebrities, and wealthy patrons. He promoted a collection of personalities known as Warhol superstars, and is credited with coining the widely used expression "15 minutes of fame." In the late 1960s, he managed and produced the experimental rock band The Velvet Underground and founded Interview magazine. He authored numerous books, including The Philosophy of Andy Warhol and Popism: The Warhol Sixties. He lived openly as a gay man before the gay liberation movement. After gallbladder surgery, Warhol died of Cardiac arrhythmia in February 1987 at the ag

['[20]', '[21]']
